<a href="https://colab.research.google.com/github/sayan0506/Face-Recognition-pipeline-using-SOTA-ArcFace/blob/main/Face_recognition_using_arcface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Face recognition Pipeline design using arcface**

This notebook is inspired from the paper [ArcFace: Additive Angular Margin Loss for Deep Face Recognition](https://arxiv.org/pdf/1801.07698v3.pdf)

Git repo: [Insightface_Pytorch](https://github.com/TreB1eN/InsightFace_Pytorch)

## **Import Dependencies**

In [47]:
from pathlib import Path
import numpy as np
import cv2

# data pipeline
# helps to create dict where, use keys as atribute
from easydict import EasyDict as edict

from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader

# ensures loading truncated images 
ImageFile.LOAD_TRUNCATED_IMAGES = True

# model build
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
import torch.nn.functional as F
import torch
from collections import namedtuple
import math
import pdb

## **Mount Drive**

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## **Dataset Load**

Load the avengers face dataset
* Source: [Kaggle](https://www.kaggle.com/rawatjitesh/avengers-face-recognition)
* Drive link: [Dataset](https://drive.google.com/drive/folders/1VYuEXVOzUtd7fOaaLv7oW4YwGYbvh80w?usp=sharing)

**Defining Configuration Dictionary, which will contain the important metadata regarding the ML pipeline**

In [28]:
# config edict dictionary initialize
config = edict()

In [29]:
# avengers face dataset path
dataset_path = "/content/gdrive/MyDrive/Kaggle Avengers face dataset/Marvels_face_dataset"
# add dataset path to config
config.imgs_folder = dataset_path

**Create train dataset from the folder**

In [30]:
def get_train_dataset(imgs_folder):
  # create torchvision transforms object for train data
  train_transform = trans.Compose([
                                   trans.RandomHorizontalFlip(), # random horizonttal flip of faces
                                   trans.ToTensor(), # convert img to torch tensor
                                   trans.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5]) # normalize
  ])
  
  # fetch and transform the images using torchvision transforms to create dataset
  # from the image folder
  ds = ImageFolder(imgs_folder, train_transform)
  class_num = ds[-1][1] + 1 # total class is the index of last folder + 1
  return ds, class_num

# create train dataset
train_ds, train_class_num = get_train_dataset(config.imgs_folder)

In [31]:
print(f'Train Dataset info {train_ds},\ncontains {train_class_num} different identities!')
print(f'Classes available in the dataset \n{train_ds.classes},\nhaving class ids {set(train_ds.targets)} respectively!')

Train Dataset info Dataset ImageFolder
    Number of datapoints: 274
    Root location: /content/gdrive/MyDrive/Kaggle Avengers face dataset/Marvels_face_dataset
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           ),
contains 5 different identities!
Classes available in the dataset 
['chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson'],
having class ids {0, 1, 2, 3, 4} respectively!


#### **Define Pytorch DataLoader**

Define the train_loader function, where we can pass dataset mode(as 'vgg' or 'ms1m') through configuration dictionary.


In [40]:
# define the train_loader function, where we can pass dataset mode(as 'vgg' or 'ms1m') through configuration dictionary
def get_train_loader(ds, class_num, config):
  if config.data_mode in ['ms1m','vgg']:
    # create train_loader, by deafult shuffles the datapoints
    train_loader = DataLoader(ds, batch_size=config.batch_size, shuffle = True, pin_memory = config.pin_memory,
                              num_workers = config.num_workers)
    return train_loader, train_class_num

**Defining metadata parameters to config**

In [41]:
# initializing data_mode as 'ms1m' dataset
config.data_mode = 'ms1m'
# initializing batch_size to 64
config.batch_size = 64
# pin_memory status is set to True, which enables to load samples of data to device(GPU), spped-up the training
config.pin_memory = True
# initializing the number of workers to 4
config.num_workers = 4

**Create the train dataloader**

In [43]:
train_loader, _ = get_train_loader(train_ds, train_class_num, config)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## **ArcFace Model Build**

There are Backbone, Mobilefacenet models for generating face embedding. But, We are considering to use only Mobilefacenet, as it's a light-weight model.

**Define Flatten Block**

In [55]:
class Flatten(Module):
  def forward(self, input):
    return input.view(input.size(0), -1)

**Define a Convolution Block**

In [48]:
# defining torch deafult sub-class api
# PReLU - Parameterized ReLU used to handle the variations(parameterize activation @ x< 0) in lower level layers 
class Conv_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Conv_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
        self.prelu = PReLU(out_c)
    
    # forward prop
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.prelu(x)
        return x

**Define Linear Block**

In [49]:
class Linear_block(Module):
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(Linear_block, self).__init__()
        self.conv = Conv2d(in_c, out_channels=out_c, kernel_size=kernel, groups=groups, stride=stride, padding=padding, bias=False)
        self.bn = BatchNorm2d(out_c)
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x

**Define Depthwise Seperable Convolution Block**

In [50]:
class Depth_Wise(Module):
     def __init__(self, in_c, out_c, residual = False, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=1):
        super(Depth_Wise, self).__init__()
        self.conv = Conv_block(in_c, out_c=groups, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.conv_dw = Conv_block(groups, groups, groups=groups, kernel=kernel, padding=padding, stride=stride)
        self.project = Linear_block(groups, out_c, kernel=(1, 1), padding=(0, 0), stride=(1, 1))
        self.residual = residual
     def forward(self, x):
        if self.residual:
            short_cut = x
        x = self.conv(x)
        x = self.conv_dw(x)
        x = self.project(x)
        if self.residual:
            output = short_cut + x
        else:
            output = x
        return output

**Define Residual Block**

In [51]:
class Residual(Module):
    def __init__(self, c, num_block, groups, kernel=(3, 3), stride=(1, 1), padding=(1, 1)):
        super(Residual, self).__init__()
        modules = []
        for _ in range(num_block):
            modules.append(Depth_Wise(c, c, residual=True, kernel=kernel, padding=padding, stride=stride, groups=groups))
        self.model = Sequential(*modules)
    def forward(self, x):
        return self.model(x)

#### **Define Mobilefacenet Model**

In [56]:
class MobileFaceNet(Module):
    def __init__(self, embedding_size):
        super(MobileFaceNet, self).__init__()
        self.conv1 = Conv_block(3, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_dw = Conv_block(64, 64, kernel=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        self.conv_23 = Depth_Wise(64, 64, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=128)
        self.conv_3 = Residual(64, num_block=4, groups=128, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_34 = Depth_Wise(64, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=256)
        self.conv_4 = Residual(128, num_block=6, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_45 = Depth_Wise(128, 128, kernel=(3, 3), stride=(2, 2), padding=(1, 1), groups=512)
        self.conv_5 = Residual(128, num_block=2, groups=256, kernel=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv_6_sep = Conv_block(128, 512, kernel=(1, 1), stride=(1, 1), padding=(0, 0))
        self.conv_6_dw = Linear_block(512, 512, groups=512, kernel=(7,7), stride=(1, 1), padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(512, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size)
    
    def forward(self, x):
        out = self.conv1(x)

        out = self.conv2_dw(out)

        out = self.conv_23(out)

        out = self.conv_3(out)
        
        out = self.conv_34(out)

        out = self.conv_4(out)

        out = self.conv_45(out)

        out = self.conv_5(out)

        out = self.conv_6_sep(out)

        out = self.conv_6_dw(out)

        out = self.conv_6_flatten(out)

        out = self.linear(out)

        out = self.bn(out)
        return l2_norm(out)

**Initialize the Mobilefacenet model**

In [59]:
# call model object
mobilefacenet_model = MobileFaceNet(512)

print(f'Model summary:\n{mobilefacenet_model}')

Model summary:
MobileFaceNet(
  (conv1): Conv_block(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (conv2_dw): Conv_block(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu): PReLU(num_parameters=64)
  )
  (conv_23): Depth_Wise(
    (conv): Conv_block(
      (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=128)
    )
    (conv_dw): Conv_block(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=128, bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, t